# MODEL 3 1-3-2
### Contains only fixed-before-conditional trials without intervening BLs
### Combines A & C trials into single regressor
### Accounts for last three noisy volumes in Lvl 1 analysis (FSL ROI -- ExtractROI)

## Compares activation for regions of interest using binarized masks:
### Hippocampus (FS labels: hippocampus [17, 53])
### Dorsal caudate (hand-drawn by Mandy)
### Putamen (FS labels: putamen [12, 51])

In [1]:
import os
from glob import glob
import nibabel as nb
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette('muted')
sns.set_style("white")

sids = ['WMAZE_001', 'WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006', 
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012', 
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',  
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
#sids = ['WMAZE_001']

proj_dir = '/home/data/madlab/data/mri/wmaze'
 
mask_filenames = []
cope_files = []

for SID in sids:
    mask_filenames_glob = glob(proj_dir + '/roi_analysis/MRthesis/fb4c_2/mask/anat_masks/_subject_id_' 
                               + SID + '/_anatmask_xfm*/*')
    mask_filenames_glob.sort()
    mask_filenames.append(mask_filenames_glob)
        
    subjcopes_glob = glob(proj_dir + '/scndlvl/wmaze_MRthesis/fixed_before_conditional/model3_1-3-2/' 
                          + SID + '/fixedfx/cope_*')
    subjcopes_glob.sort()
    cope_files.append(subjcopes_glob)
    if len(cope_files[-1]) == 0 or len(cope_files[-1]) != 5:
        print(SID)

## Cell to double-check the array indexing

In [ ]:
#change first index of cope_files to indicate participant index in sids array
for i, curr_mask in enumerate(mask_filenames[0]):
    print(i, mask_filenames[0][i].split('/')[-1][:-7])

In [ ]:
for i, curr_cope in enumerate(cope_files[0]):
    print(i, cope_files[0][i].split('/')[-1][5:-7]) 

### Use binarized mask to obtain activation in left & right hemisphere for each region of interest

In [2]:
all_data = {}
weighted_data = {}
all_voxs = {'hp':[], 'mpfc':[], 'caud':[], 'put':[]}

for r in ['hp', 'mpfc', 'caud', 'put']:
    for acc in ['corr', 'incorr']:
        all_data['{0}'.format(r)] = {'corr':[], 'incorr':[]}

for i in range(len(sids)):
    #hpc_fb4c['subjid'].append(sids[i]) 
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][11])
    lh_mpfc_img = nb.load(mask_filenames[i][4])
    rh_mpfc_img = nb.load(mask_filenames[i][13])
    lh_caud_img = nb.load(mask_filenames[i][7])
    rh_caud_img = nb.load(mask_filenames[i][8])
    lh_put_img = nb.load(mask_filenames[i][16])
    rh_put_img = nb.load(mask_filenames[i][17])

    for key in all_voxs:
        voxs = eval('sum(sum(sum(lh_{0}_img.get_data() + rh_{0}_img.get_data())))'.format(key))
        all_voxs['{0}'.format(key)].append(voxs)
    
    all_before_B_corr_img = nb.load(cope_files[i][0])
    all_before_B_incorr_img = nb.load(cope_files[i][1])    

    for key in all_data:
        for acc in ['corr', 'incorr']:
            lh_data = eval('np.mean(all_before_B_{0}_img.get_data()[lh_{1}_img.get_data() > 0.])'.format(acc,key))            
            rh_data = eval('np.mean(all_before_B_{0}_img.get_data()[rh_{1}_img.get_data() > 0.])'.format(acc,key))
            all_data['{0}'.format(key)]['{0}'.format(acc)].append((lh_data + rh_data)/2.)
                        
for key in all_data:
    exec('{0}_fb4c = all_data["{0}"]'.format(key))
    exec('{0}_voxs = np.array(all_voxs["{0}"])'.format(key))
    for acc in ['corr', 'incorr']:
        exec('weighted_data["{0}_{1}"] = {0}_fb4c["{1}"] * {0}_voxs'.format(key, acc))
        
weighted_data_df = pd.DataFrame(weighted_data)
all_voxs_df = pd.DataFrame(all_voxs)

In [3]:
#print hp_fb4c['corr'][0], hp_voxs[0]
#print weighted_data['hp_corr'] 

x = sum(sum(sum(all_before_B_corr_img.get_data())))
print hp_voxs[0]/x

0.00598125448458


In [4]:
all_voxs_df

,caud,hp,mpfc,put
0,265.0,523.0,468.0,731.0
1,267.0,562.0,639.0,791.0
2,278.0,429.0,480.0,559.0
3,241.0,464.0,497.0,767.0
4,239.0,443.0,463.0,628.0
5,300.0,517.0,582.0,631.0
6,227.0,499.0,448.0,703.0
7,320.0,525.0,553.0,686.0
8,317.0,466.0,474.0,708.0
9,271.0,353.0,534.0,464.0


In [7]:
print all_voxs_df.mean()
print ""
print all_voxs_df.std()

caud    279.70
hp      483.40
mpfc    502.65
put     650.35
dtype: float64

caud    30.578802
hp      50.391520
mpfc    64.231715
put     82.926267
dtype: float64


In [ ]:
print stats.ttest_rel(all_voxs_df['hp'], all_voxs_df['mpfc'])

In [ ]:
from math import sqrt
for key in ['hp', 'mpfc']:
    print "Mean {0}: ".format(key), np.mean(all_voxs_df['{0}'.format(key)])
    print "STD {0}: ".format(key), np.std(all_voxs_df['{0}'.format(key)])
    print ""
print "Paired-Samples t-Test:", stats.ttest_rel(all_voxs_df['hp'], all_voxs_df['mpfc'])
cohens_d = ((np.average(all_voxs_df['hp']) - np.average(all_voxs_df['mpfc'])) 
            / (sqrt((np.std(all_voxs_df['hp'], ddof = 1)) ** 2 + np.std( all_voxs_df['mpfc'], ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d

N = 2
conditions = ['HPC', 'mPFC']
hp_allsubjs = [all_voxs_df['hp'], all_voxs_df['mpfc']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (6,5))
ax0 = sns.boxplot(data = hp_allsubjs, width = 0.3, linewidth = 2)
ax2 = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticks(ind)
ax.set_xticklabels(conditions)
plt.savefig("/home/arenf001/voxels.png", dpi = 1080)
plt.show()

In [ ]:
weighted_data_df

## Hippocampus

In [ ]:
from math import sqrt
for acc in ['corr', 'incorr']:
    print "Mean HPC_{0}: ".format(acc), np.mean(weighted_data_df['hp_{0}'.format(acc)])
    print "STD HPC {0}: ".format(acc), np.std(weighted_data_df['hp_{0}'.format(acc)])
    print ""
print "Paired-Samples t-Test:", stats.ttest_rel(weighted_data_df['hp_corr'], weighted_data_df['hp_incorr'])
cohens_d = ((np.average(weighted_data_df['hp_corr']) - np.average(weighted_data_df['hp_incorr'])) 
            / (sqrt((np.std(weighted_data_df['hp_corr'], ddof = 1)) 
            ** 2 + np.std(weighted_data_df['hp_incorr'], ddof = 1) ** 2) / 2))
print "Cohen's d = ", cohens_d

N = 2
conditions = ['Correct', 'Incorrect']
hp_allsubjs = [weighted_data_df['hp_corr'], weighted_data_df['hp_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (6,5))
ax0 = sns.boxplot(data = hp_allsubjs, color = "#278fea", width = 0.3)
ax2 = sns.swarmplot(data = hp_allsubjs, color='.25')
ax.set_xticks(ind)
ax.set_xticklabels(conditions)
ax.set_ylabel("Arbitrary units")
ax.set_ylim(-4000,3000)
ax.set_title("Hippocampus")
plt.savefig("/home/arenf001/weightedHPC.png", dpi = 1080)
plt.show()

## mPFC

In [ ]:
for acc in ['corr', 'incorr']:
    print "Mean mPFC_{0}: ".format(acc), np.mean(weighted_data_df['mpfc_{0}'.format(acc)])
    print "STD mPFC {0}: ".format(acc), np.std(weighted_data_df['mpfc_{0}'.format(acc)])
    print ""
print "Paired-Samples t-Test:", stats.ttest_rel(weighted_data_df['mpfc_corr'], weighted_data_df['mpfc_incorr'])
N = 2
conditions = ['Correct', 'Incorrect']
mPFC_allsubjs = [weighted_data_df['mpfc_corr'], weighted_data_df['mpfc_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (6,5))
ax0 = sns.boxplot(data = mPFC_allsubjs, color = "#f97401", width = 0.3)
ax2 = sns.swarmplot(data = mPFC_allsubjs, color='.25')
ax.set_xticks(ind)
ax.set_xticklabels(conditions)
ax.set_ylabel("Arbitrary units")
ax.set_title("Medial PFC")
plt.savefig("/home/arenf001/weightedMPFC.png", dpi = 1080)
plt.show()

## Caudate

In [ ]:
for acc in ['corr', 'incorr']:
    print "Mean Caudate_{0}: ".format(acc), np.mean(weighted_data_df['caud_{0}'.format(acc)])
    print "STD Caudate {0}: ".format(acc), np.std(weighted_data_df['caud_{0}'.format(acc)])
    print ""
print "Paired-Samples t-Test:", stats.ttest_rel(weighted_data_df['caud_corr'], weighted_data_df['caud_incorr'])
N = 2
conditions = ['Correct', 'Incorrect']
caud_allsubjs = [weighted_data_df['caud_corr'], weighted_data_df['caud_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (6,5))
ax0 = sns.boxplot(data = caud_allsubjs, color = "#f9f96d", width = 0.3)
ax2 = sns.swarmplot(data =  caud_allsubjs, color='.25')
ax.set_xticks(ind)
ax.set_xticklabels(conditions)
ax.set_ylabel("Arbitrary units")
ax.set_title("Caudate")
plt.savefig("/home/arenf001/weightedCAUD.png", dpi = 1080)
plt.show()

## Putamen

In [ ]:
for acc in ['corr', 'incorr']:
    print "Mean Putamen {0}: ".format(acc), np.mean(weighted_data_df['put_{0}'.format(acc)])
    print "STD Putamen {0}: ".format(acc), np.std(weighted_data_df['put_{0}'.format(acc)])
    print ""
print "Paired-Samples t-Test:", stats.ttest_rel(weighted_data_df['put_corr'], weighted_data_df['put_incorr'])
N = 2
conditions = ['Correct', 'Incorrect']
put_allsubjs = [weighted_data_df['put_corr'], weighted_data_df['put_incorr']]
ind = np.arange(N)
fig, ax = plt.subplots(figsize = (6,5))
ax0 = sns.boxplot(data = put_allsubjs, color = "#c34aef", width = 0.3)
ax2 = sns.swarmplot(data =  put_allsubjs, color='.25')
ax.set_xticks(ind)
ax.set_xticklabels(conditions)
ax.set_ylabel("Arbitrary units")
ax.set_title("Putamen")
plt.savefig("/home/arenf001/weightedPUT.png", dpi = 1080)
plt.show()